In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import pyspark

In [3]:
import os
print('SPARK_HOME', os.environ.get("SPARK_HOME"))
print('HADOOP_HOME', os.environ.get("HADOOP_HOME"))
print('JAVA_HOME', os.environ.get("JAVA_HOME"))
print(os.environ['PATH'])

SPARK_HOME C:\spark-3.1.2-bin-hadoop3.2\spark-3.1.2-bin-hadoop3.2
HADOOP_HOME C:\winutils
JAVA_HOME C:\Java\jdk-15.0.1
C:\Users\adrien\Anaconda3;C:\Users\adrien\Anaconda3\Library\mingw-w64\bin;C:\Users\adrien\Anaconda3\Library\usr\bin;C:\Users\adrien\Anaconda3\Library\bin;C:\Users\adrien\Anaconda3\Scripts;C:\Users\adrien\Anaconda3\bin;C:\Users\adrien\Anaconda3\condabin;C:\Users\adrien\Anaconda3;C:\Users\adrien\Anaconda3\Library\mingw-w64\bin;C:\Users\adrien\Anaconda3\Library\usr\bin;C:\Users\adrien\Anaconda3\Library\bin;C:\Users\adrien\Anaconda3\Scripts;C:\WINDOWS\System32;C:\Users\adrien\AppData\Local\Microsoft\WindowsApps;C:\spark-3.1.2-bin-hadoop3.2\spark-3.1.2-bin-hadoop3.2\bin;C:\Java\jdk-15.0.1\bin;C:\winutils\bin;C:\Windows\System32;.;C:\Users\adrien\Anaconda3\lib\site-packages\numpy\.libs


In [4]:
## spark imports
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import *

In [5]:
## standard imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import format_number, when, col, array, udf, lit
import pandas as pd
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.types import *
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

In [6]:
%matplotlib inline

sns.set_color_codes("pastel")
plt.rcParams["figure.figsize"] = [20, 8]

In [7]:
import findspark
findspark.init()

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Chicago_crime_prediction").config('spark.sql.execution.arrow.pyspark.enabled', True).config('spark.sql.session.timeZone', 'UTC').config('spark.driver.memory','32G').config("spark.driver.cores", '3').getOrCreate()

In [9]:
from pyspark.sql.types import  (StructType, 
                                StructField, 
                                DateType, 
                                BooleanType,
                                DoubleType,
                                IntegerType,
                                StringType,
                               TimestampType)


crimes_schema = StructType([StructField("ID", StringType(), True),
                            StructField("CaseNumber", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("PrimaryType", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("LocationDescription", StringType(), True ),
                            StructField("Arrest", StringType(), True),
                            StructField("Domestic", StringType(), True),
                            StructField("Beat", StringType(), True),
                            StructField("District", StringType(), True),
                            StructField("Ward", StringType(), True),
                            StructField("CommunityArea", StringType(), True),
                            StructField("FBICode", StringType(), True ),
                            StructField("XCoordinate", DoubleType(), True),
                            StructField("YCoordinate", DoubleType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("UpdatedOn", DateType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [10]:
crimes = spark.read.csv("Crimes_-_2001_to_Present.csv",
                       header = True, 
                        schema = crimes_schema)

In [11]:
print(" The crimes dataframe has {} records".format(crimes.count()))

 The crimes dataframe has 7393744 records


In [12]:
crimes.show()

+--------+----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+--------------------+
|      ID|CaseNumber|                Date|               Block|IUCR|       PrimaryType|         Description| LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year|UpdatedOn|    Latitude|    Longitude|            Location|
+--------+----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+--------------------+
|10224738|  HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     009|  12|           61|

### Data preprocessing - Spark 

In [13]:
dataset = crimes.filter((crimes['Year'] >2009) & (crimes['Year'] <= 2021))

In [14]:
dataset = dataset.drop('ID')
dataset = dataset.drop('Case Number')
dataset = dataset.drop('Updated On')
dataset = dataset.drop('IUCR')
dataset = dataset.drop('Y Coordinate')
dataset = dataset.drop('Location')
dataset = dataset.drop('Description')

In [15]:
dataset.show()

+----------+--------------------+--------------------+------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+
|CaseNumber|                Date|               Block|       PrimaryType| LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year|UpdatedOn|    Latitude|    Longitude|
+----------+--------------------+--------------------+------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+
|  HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|           BATTERY|           RESIDENCE| false|    true|0924|     009|  12|           61|    08B|  1165074.0|  1875917.0|2015|     null|41.815117282|-87.669999562|
|  HY411615|09/04/2015 11:30:...| 008XX N CENTRAL AVE|             THEFT|             CTA BUS| false|   false|1511|     015|  29

### Preprocessing Primary types
 

In [16]:
primaryCount = dataset.groupby('PrimaryType').count()
primaryCount.orderBy('count', ascending=False).show()

+--------------------+------+
|         PrimaryType| count|
+--------------------+------+
|               THEFT|737239|
|             BATTERY|604218|
|     CRIMINAL DAMAGE|356975|
|           NARCOTICS|268698|
|             ASSAULT|224246|
|       OTHER OFFENSE|202662|
|  DECEPTIVE PRACTICE|191405|
|            BURGLARY|182714|
| MOTOR VEHICLE THEFT|145267|
|             ROBBERY|127007|
|   CRIMINAL TRESPASS| 81519|
|   WEAPONS VIOLATION| 55675|
|OFFENSE INVOLVING...| 27153|
|PUBLIC PEACE VIOL...| 25747|
|        PROSTITUTION| 14972|
| CRIM SEXUAL ASSAULT| 13695|
|INTERFERENCE WITH...| 12799|
|         SEX OFFENSE| 12553|
|            HOMICIDE|  6634|
|               ARSON|  5349|
+--------------------+------+
only showing top 20 rows



In [122]:
print('Distinct primary types '+ str(primaryCount.distinct().count()))

Distinct primary types 35


### Supression des crimes non significant

In [15]:
dataset = dataset.filter((dataset['PrimaryType']!='STALKING')&
                             (dataset['PrimaryType']!='OBSCENITY')&
                             (dataset['PrimaryType']!='NON-CRIMINAL (SUBJECT SPECIFIED)')&
                             (dataset['PrimaryType']!='GAMBLING')&
                             (dataset['PrimaryType']!='NON - CRIMINAL')&
                             (dataset['PrimaryType']!='LIQUOR LAW VIOLATION')&
                             (dataset['PrimaryType']!='PUBLIC INDECENCY')&
                             (dataset['PrimaryType']!='HUMAN TRAFFICKING')&
                             (dataset['PrimaryType']!='INTIMIDATION')&
                             (dataset['PrimaryType']!='CONCEALED CARRY LICENSE VIOLATION')&
                             (dataset['PrimaryType']!='NON-CRIMINAL')&
                             (dataset['PrimaryType']!='OTHER NARCOTIC VIOLATION'))

### Merge des crimes similaires

In [16]:
dataset = dataset.withColumn("PrimaryType", \
                               when((dataset["PrimaryType"] == 'CRIM SEXUAL ASSAULT') | \
                                    (dataset["PrimaryType"] == 'PROSTITUTION') ,'SEX OFFENSE').otherwise(dataset['PrimaryType']))
dataset=dataset.withColumn("PrimaryType", \
                               when((dataset["PrimaryType"] == 'WEAPONS VIOLATION') | \
                                    (dataset["PrimaryType"] == 'INTERFERENCE WITH PUBLIC OFFICER') ,'PUBLIC PEACE VIOLATION').otherwise(dataset['PrimaryType']))
dataset=dataset.withColumn("PrimaryType", \
                               when((dataset["PrimaryType"] == 'ARSON') ,'CRIMINAL TRESPASS').otherwise(dataset['PrimaryType']))

In [125]:
primaryCount = dataset.groupby('PrimaryType').count()
primaryCount.orderBy('count', ascending=False).show()

+--------------------+------+
|         PrimaryType| count|
+--------------------+------+
|               THEFT|737239|
|             BATTERY|604218|
|     CRIMINAL DAMAGE|356975|
|           NARCOTICS|268698|
|             ASSAULT|224246|
|       OTHER OFFENSE|202662|
|  DECEPTIVE PRACTICE|191405|
|            BURGLARY|182714|
| MOTOR VEHICLE THEFT|145267|
|             ROBBERY|127007|
|PUBLIC PEACE VIOL...| 94221|
|   CRIMINAL TRESPASS| 86868|
|         SEX OFFENSE| 41220|
|OFFENSE INVOLVING...| 27153|
|            HOMICIDE|  6634|
|CRIMINAL SEXUAL A...|  3451|
|          KIDNAPPING|  2380|
|           RITUALISM|     1|
+--------------------+------+



In [126]:
print('Distinct primary types after preprocessing '+ str(primaryCount.distinct().count()))

Distinct primary types after preprocessing 18


In [17]:
dataset.show()

+----------+--------------------+--------------------+------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+
|CaseNumber|                Date|               Block|       PrimaryType| LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year|UpdatedOn|    Latitude|    Longitude|
+----------+--------------------+--------------------+------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+
|  HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|           BATTERY|           RESIDENCE| false|    true|0924|     009|  12|           61|    08B|  1165074.0|  1875917.0|2015|     null|41.815117282|-87.669999562|
|  HY411615|09/04/2015 11:30:...| 008XX N CENTRAL AVE|             THEFT|             CTA BUS| false|   false|1511|     015|  29

### Supression des valeurs NaN

In [18]:
#dataset=dataset.na.drop()
dataset.show()

+----------+--------------------+--------------------+------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+
|CaseNumber|                Date|               Block|       PrimaryType| LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year|UpdatedOn|    Latitude|    Longitude|
+----------+--------------------+--------------------+------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+
|  HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|           BATTERY|           RESIDENCE| false|    true|0924|     009|  12|           61|    08B|  1165074.0|  1875917.0|2015|     null|41.815117282|-87.669999562|
|  HY411615|09/04/2015 11:30:...| 008XX N CENTRAL AVE|             THEFT|             CTA BUS| false|   false|1511|     015|  29

### Feature Engineering

In [19]:
selected_features = [
 'LocationDescription',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'CommunityArea',
 'FBICode',
 'Year']

In [20]:
features_df = dataset.select(selected_features)
features_df.printSchema()

root
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- CommunityArea: string (nullable = true)
 |-- FBICode: string (nullable = true)
 |-- Year: integer (nullable = true)



In [21]:
feature_level_count_dic = []

for feature in selected_features:
    print('Analysing %s' % feature)
    levels_list_df = features_df.select(feature).distinct()
    feature_level_count_dic.append({'feature': feature, 'level_count': levels_list_df.count()})

Analysing LocationDescription
Analysing Arrest
Analysing Domestic
Analysing Beat
Analysing District
Analysing Ward
Analysing CommunityArea
Analysing FBICode
Analysing Year


In [22]:
pd.DataFrame(feature_level_count_dic).sort_values(by='level_count', ascending=False)

,feature,level_count
3,Beat,303
0,LocationDescription,199
6,CommunityArea,79
5,Ward,51
7,FBICode,25
4,District,24
8,Year,12
1,Arrest,2
2,Domestic,2


### Preparing Model 

In [23]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
df_dates_features = dataset.na.drop(subset=selected_features)

In [24]:
dataset.printSchema()

root
 |-- CaseNumber: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- PrimaryType: string (nullable = true)
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- CommunityArea: string (nullable = true)
 |-- FBICode: string (nullable = true)
 |-- XCoordinate: double (nullable = true)
 |-- YCoordinate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- UpdatedOn: date (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [60]:
dataset.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



Indexation des features 

In [25]:
for feature in feature_level_count_dic:
    indexer = StringIndexer(inputCol=feature['feature'], outputCol='%s_indexed' % feature['feature'])
    print('Fitting feature "%s"' % feature['feature'])
    model = indexer.fit(df_dates_features)
    print('Transforming "%s"' % feature['feature'])
    df_dates_features = model.transform(df_dates_features)

Fitting feature "LocationDescription"
Transforming "LocationDescription"
Fitting feature "Arrest"
Transforming "Arrest"
Fitting feature "Domestic"
Transforming "Domestic"
Fitting feature "Beat"
Transforming "Beat"
Fitting feature "District"
Transforming "District"
Fitting feature "Ward"
Transforming "Ward"
Fitting feature "CommunityArea"
Transforming "CommunityArea"
Fitting feature "FBICode"
Transforming "FBICode"
Fitting feature "Year"
Transforming "Year"


In [26]:
## String-index des features:
response_indexer = StringIndexer(inputCol='PrimaryType', outputCol='PrimaryType_indexed')
response_model = response_indexer.fit(df_dates_features)
df_dates_features = response_model.transform(df_dates_features)

In [27]:
df_dates_features.show(1)

+----------+--------------------+---------------+-----------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+---------------------------+--------------+----------------+------------+----------------+------------+---------------------+---------------+------------+-------------------+
|CaseNumber|                Date|          Block|PrimaryType|LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year|UpdatedOn|    Latitude|    Longitude|LocationDescription_indexed|Arrest_indexed|Domestic_indexed|Beat_indexed|District_indexed|Ward_indexed|CommunityArea_indexed|FBICode_indexed|Year_indexed|PrimaryType_indexed|
+----------+--------------------+---------------+-----------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------+------------+-------------+---------------------------+----------

Vectorisons nos variables

Nous utilisons a vecteur assembleur :

In [28]:
indexed_features = ['%s_indexed' % fc['feature'] for fc in feature_level_count_dic]
indexed_features

['LocationDescription_indexed',
 'Arrest_indexed',
 'Domestic_indexed',
 'Beat_indexed',
 'District_indexed',
 'Ward_indexed',
 'CommunityArea_indexed',
 'FBICode_indexed',
 'Year_indexed']

In [29]:
assembler = VectorAssembler(inputCols=indexed_features, outputCol='features')
vectorized_df_dates = assembler.transform(df_dates_features)

In [30]:
vectorized_df_dates.select('features').take(1)

[Row(features=DenseVector([1.0, 0.0, 1.0, 119.0, 8.0, 35.0, 18.0, 1.0, 8.0]))]

### Entrainement du modèle

Séparation

In [31]:
train, test = vectorized_df_dates.randomSplit([0.6, 0.4])

In [32]:
from pyspark.ml.classification import LogisticRegression

### Régression logistique 

In [45]:
logisticRegression = LogisticRegression(labelCol='PrimaryType_indexed', featuresCol='features', maxIter=10, family='multinomial')

In [46]:
fittedModel = logisticRegression.fit(train)

### Model performance

In [47]:
fittedModel.summary.accuracy

0.5104731631838821

In [48]:
model_summary = fittedModel.summary
model_summary

In [49]:
fittedModel.coefficientMatrix

DenseMatrix(18, 9, [0.0155, -0.7901, -1.7199, 0.0044, 0.0855, 0.0072, 0.0094, -1.4153, ..., -0.0001, 0.0002, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0], 1)

In [50]:
print(fittedModel.coefficientMatrix)

DenseMatrix([[ 1.54938249e-02, -7.90088589e-01, -1.71993523e+00,
               4.37204467e-03,  8.54897510e-02,  7.21930038e-03,
               9.40922886e-03, -1.41530509e+00,  1.75965772e-01],
             [ 2.05344963e-02,  1.63685889e-01,  2.48594422e+00,
               2.61022128e-03,  3.96607860e-02,  7.11780011e-03,
               4.05231869e-03, -3.00755861e-01,  1.06533757e-01],
             [ 1.72886544e-03, -2.13712821e+00,  1.09879122e+00,
               3.15538000e-03,  3.78974846e-02,  1.62867200e-02,
               9.36903351e-03, -3.81702624e-01,  1.57852583e-01],
             [-3.76391763e-02,  6.07240616e+00, -2.22588250e+00,
              -3.64784076e-03, -6.77679315e-02, -2.44105277e-02,
              -1.53367351e-02, -1.46405328e-01, -2.32705085e-01],
             [ 9.67413212e-03, -5.01731714e-01,  2.39348699e+00,
              -5.23950480e-04, -9.57645593e-03, -1.50667246e-03,
               2.21320275e-03,  2.14440093e-01, -1.73990115e-02],
             [-1.246

In [51]:
print('Coefficient matrix:\nRow count = %s\nCol count = %s' % (fittedModel.coefficientMatrix.numRows, fittedModel.coefficientMatrix.numCols))

Coefficient matrix:
Row count = 18
Col count = 9


In [52]:
print('Model:\nNum Classes = %s\nNum Features = %s' % (fittedModel.numClasses, fittedModel.numFeatures))

Model:
Num Classes = 18
Num Features = 9


In [135]:
print('Training "primary_type" factor level count = %s' % train.select('PrimaryType_indexed').distinct().count())

Training "primary_type" factor level count = 18


In [53]:
vectorized_df_dates.select('features').show(2, truncate=False)

+------------------------------------------+
|features                                  |
+------------------------------------------+
|[1.0,0.0,1.0,119.0,8.0,35.0,18.0,1.0,8.0] |
|[26.0,0.0,0.0,107.0,15.0,18.0,0.0,0.0,8.0]|
+------------------------------------------+
only showing top 2 rows



In [137]:
fittedModel.numClasses

18

In [138]:
fittedModel.numFeatures

9

In [140]:
train.select('PrimaryType_indexed').distinct().count()

18

In [141]:
print(model_summary.objectiveHistory)
print()
print('Objective history size ', len(model_summary.objectiveHistory))

[2.3471699627373153, 2.047512628384678, 1.927207712023048, 1.7920399555427182, 1.7275775994154927, 1.6452940122554582, 1.630919487061892, 1.5541222709098568, 1.543171443715006, 1.528196015216986, 1.4944269735717701]

Objective history size  11


In [143]:
label_stats = {float(i):{'index': float(i)} for i in range(34)}
print(label_stats)

{0.0: {'index': 0.0}, 1.0: {'index': 1.0}, 2.0: {'index': 2.0}, 3.0: {'index': 3.0}, 4.0: {'index': 4.0}, 5.0: {'index': 5.0}, 6.0: {'index': 6.0}, 7.0: {'index': 7.0}, 8.0: {'index': 8.0}, 9.0: {'index': 9.0}, 10.0: {'index': 10.0}, 11.0: {'index': 11.0}, 12.0: {'index': 12.0}, 13.0: {'index': 13.0}, 14.0: {'index': 14.0}, 15.0: {'index': 15.0}, 16.0: {'index': 16.0}, 17.0: {'index': 17.0}, 18.0: {'index': 18.0}, 19.0: {'index': 19.0}, 20.0: {'index': 20.0}, 21.0: {'index': 21.0}, 22.0: {'index': 22.0}, 23.0: {'index': 23.0}, 24.0: {'index': 24.0}, 25.0: {'index': 25.0}, 26.0: {'index': 26.0}, 27.0: {'index': 27.0}, 28.0: {'index': 28.0}, 29.0: {'index': 29.0}, 30.0: {'index': 30.0}, 31.0: {'index': 31.0}, 32.0: {'index': 32.0}, 33.0: {'index': 33.0}}


In [54]:
# Affichage des coefficients multinomial logistic regression 
print("Intercept: " + str(fittedModel.interceptVector))

Intercept: [3.7011535257929946,2.704635199829962,2.5157821376325566,1.3596615168621424,1.2228738703735678,1.497247772838806,0.9047543364717285,1.498633891601703,0.9707348486746903,0.8580598638904492,0.27285426999765394,0.5557164008050451,-0.3100238436782924,-0.5882471358001135,-1.949568397935904,-2.6056850263623694,-3.03008550465665,-9.578497726337968]


In [56]:
# Obtention de l'objectif par itération
objectiveHistory = model_summary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

objectiveHistory:
2.3466766002513353
2.046847650840424
1.9266881255755162
1.7915751556022557
1.7270246542321832
1.6457818604643588
1.625777891557425
1.551904173552308
1.5406666469394823
1.5245030448170087
1.4911038733211073


In [57]:
print("False positive rate by label:")
for i, rate in enumerate(model_summary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(model_summary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(model_summary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(model_summary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(model_summary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

False positive rate by label:
label 0: 0.0970866442612539
label 1: 0.16978206868484333
label 2: 0.0501507708907124
label 3: 0.049720900701496026
label 4: 0.017130909533496774
label 5: 0.00012083998897335101
label 6: 0.07482752833778977
label 7: 0.022211072731132167
label 8: 0.029690567221037488
label 9: 0.03113442957254676
label 10: 0.015052619014981212
label 11: 0.004446136229614075
label 12: 0.00020917659785542718
label 13: 0.0
label 14: 0.0
label 15: 0.0
label 16: 0.0
label 17: 0.0
True positive rate by label:
label 0: 0.936635031371039
label 1: 0.4620722318291461
label 2: 0.47713196760362075
label 3: 0.8246389442231076
label 4: 0.15978370669677341
label 5: 6.592120767652464e-05
label 6: 0.4808246710137415
label 7: 0.27318890877872354
label 8: 0.3422345137821028
label 9: 0.22565977409125457
label 10: 0.7002037381521835
label 11: 0.020257543724774167
label 12: 0.013812492360347145
label 13: 0.0
label 14: 0.0
label 15: 0.0
label 16: 0.0
label 17: 0.0
Precision by label:
label 0: 0.735

In [58]:
accuracy = model_summary.accuracy
falsePositiveRate = model_summary.weightedFalsePositiveRate
truePositiveRate = model_summary.weightedTruePositiveRate
fMeasure = model_summary.weightedFMeasure()
precision = model_summary.weightedPrecision
recall = model_summary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.5104731631838821
FPR: 0.07203682694961477
TPR: 0.510473163183882
F-measure: 0.468505973300628
Precision: 0.4581589225159007
Recall: 0.510473163183882


In [154]:
train_rdd = train.select(['PrimaryType', 'PrimaryType_indexed']).distinct().orderBy('PrimaryType_indexed').rdd.map(lambda l: l.asDict())
train_rdd

PythonRDD[883] at RDD at PythonRDD.scala:53

### Predictions

In [61]:
predictions = fittedModel.transform(test)
predictions.select("prediction", "PrimaryType_indexed").show()

+----------+-------------------+
|prediction|PrimaryType_indexed|
+----------+-------------------+
|       4.0|               12.0|
|       4.0|               12.0|
|       3.0|                3.0|
|       9.0|                6.0|
|       3.0|                3.0|
|       3.0|                3.0|
|       4.0|               12.0|
|       3.0|                3.0|
|       3.0|                3.0|
|       1.0|                5.0|
|       3.0|                3.0|
|       3.0|                3.0|
|       1.0|                5.0|
|       3.0|                3.0|
|       3.0|                3.0|
|       3.0|                3.0|
|       3.0|                3.0|
|       3.0|                3.0|
|       3.0|                3.0|
|       3.0|                3.0|
+----------+-------------------+
only showing top 20 rows



### Random Forest

In [33]:
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC

In [34]:
dtc = RandomForestClassifier(labelCol="PrimaryType_indexed",featuresCol="features", numTrees = 100,maxDepth = 4,maxBins = 304)
dtc = dtc.fit(train)

In [35]:
predictions = dtc.transform(test)

In [36]:
predictions

DataFrame[CaseNumber: string, Date: string, Block: string, PrimaryType: string, LocationDescription: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, CommunityArea: string, FBICode: string, XCoordinate: double, YCoordinate: double, Year: int, UpdatedOn: date, Latitude: double, Longitude: double, LocationDescription_indexed: double, Arrest_indexed: double, Domestic_indexed: double, Beat_indexed: double, District_indexed: double, Ward_indexed: double, CommunityArea_indexed: double, FBICode_indexed: double, Year_indexed: double, PrimaryType_indexed: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [37]:
evaluator = MulticlassClassificationEvaluator(labelCol="PrimaryType_indexed", predictionCol="prediction",
 metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.5999875598681346


### Bayes Classifier

In [42]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [43]:
nb = NaiveBayes(labelCol='PrimaryType_indexed', featuresCol='features',smoothing=1.0, modelType="multinomial")

In [44]:
# Fit le model
model = nb.fit(train)

In [51]:
model.setFeaturesCol("features")

NaiveBayesModel: uid=NaiveBayes_a9b77b17559b, modelType=multinomial, numClasses=17, numFeatures=9

In [52]:
model.getSmoothing()

1.0

In [53]:
model.getLabelCol()

'PrimaryType_indexed'

In [54]:
model.getPredictionCol()

'prediction'

In [58]:
predictions.printSchema()

root
 |-- CaseNumber: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- PrimaryType: string (nullable = true)
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- CommunityArea: string (nullable = true)
 |-- FBICode: string (nullable = true)
 |-- XCoordinate: double (nullable = true)
 |-- YCoordinate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- UpdatedOn: date (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- LocationDescription_indexed: double (nullable = false)
 |-- Arrest_indexed: double (nullable = false)
 |-- Domestic_indexed: double (nullable = false)
 |-- Beat_indexed: double (nullable = false)
 |-- District_indexed: double (nullable = false)
 |-- Ward_indexed: 

In [60]:
evaluator = MulticlassClassificationEvaluator(labelCol="PrimaryType_indexed", predictionCol="prediction",
 metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.3439429697869872


In [61]:
# Prediction sur l'ensemble test
predictions = model.transform(test)
predictions.select("PrimaryType_indexed", "prediction", "probability").show()

+-------------------+----------+--------------------+
|PrimaryType_indexed|prediction|         probability|
+-------------------+----------+--------------------+
|                6.0|       8.0|[7.95322027935241...|
|                2.0|       5.0|[3.11339454214646...|
|                1.0|      11.0|[1.63270339488149...|
|               12.0|      12.0|[5.04594864705636...|
|                3.0|       3.0|[1.36974332620380...|
|                3.0|       8.0|[2.63126253589758...|
|               12.0|       8.0|[1.48829282504429...|
|               15.0|      12.0|[6.91556902154661...|
|               12.0|       8.0|[1.12657052546990...|
|                3.0|       3.0|[1.41578039688462...|
|               12.0|      13.0|[1.42615323702761...|
|               10.0|       8.0|[7.15256251752953...|
|               10.0|      12.0|[4.50895197259987...|
|               12.0|      12.0|[7.61212490520277...|
|                3.0|       3.0|[1.86579783925165...|
|                3.0|       

### MLP

In [38]:
# create the trainer and set its parameters
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

layers = [4,5,5,18]
trainer = MultilayerPerceptronClassifier(labelCol='PrimaryType_indexed',featuresCol='features',maxIter=100, layers=layers, blockSize=128, seed=1234)

In [39]:
# train the model
model = trainer.fit(train)

In [41]:
#compute accuracy on the test set
result = model.transform(test)
result

DataFrame[CaseNumber: string, Date: string, Block: string, PrimaryType: string, LocationDescription: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, CommunityArea: string, FBICode: string, XCoordinate: double, YCoordinate: double, Year: int, UpdatedOn: date, Latitude: double, Longitude: double, LocationDescription_indexed: double, Arrest_indexed: double, Domestic_indexed: double, Beat_indexed: double, District_indexed: double, Ward_indexed: double, CommunityArea_indexed: double, FBICode_indexed: double, Year_indexed: double, PrimaryType_indexed: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [3]:
evaluator = MulticlassClassificationEvaluator(labelCol="PrimaryType_indexed", predictionCol="prediction",
 metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = " + str(accuracy))